<a href="https://colab.research.google.com/github/paulorfbr/transformers_course/blob/encoder_pytorch_layer/Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class MultiheadAttention(nn.Module):
     def __init__(self, d_k, d_model, n_heads):
         super().__init__()
         #assume d_v = d_k
         self.d_k = d_k
         self.d_model = d_model
         self.n_heads = n_heads
         self.key = nn.Linear(d_model, d_k * n_heads)
         self.query = nn.Linear(d_model, d_k * n_heads)
         self.value = nn.Linear(d_model, d_k * n_heads)
         # final linear layer
         self.fc = nn.Linear(d_k * n_heads, d_model)

     def forward(self, q, k, v, mask=None):
         q = self.query(q) # N x T x (hd_k)
         k = self.key(k)   # N x T x (hd_k)
         v = self.value(v) # N x T x (hd_v)

         N = q.shape[0]
         T = q.shape[1]

         # change the shape to:
         # (N, T, h, d_k) -> (N, h, T, d_k)
         # in order for matrix multiply to work properly
         q = q.view(N, T, self.n_heads, self.d_k).transpose(1, 2)
         k = k.view(N, T, self.n_heads, self.d_k).transpose(1, 2)
         v = v.view(N, T, self.n_heads, self.d_k).transpose(1, 2)

         # (N, h, T, d_k) x (N, h, d_k, T) --> (N, h, T, T)
         attn_scores = q @ k.transpose(-2, -1) / math.sqrt(self.d_k)
         if mask is not None:
            attn_scores = attn_scores.masked_fill(
                     mask[:, None, None, :] == 0, float('-inf'))
         attn_weights = F.softmax(attn_scores, dim=-1)

         # compute attention-weighted values
         # (N, h, T, T) x (N, h, T, d_k) --> (N, h, T, d_k)
         A = attn_weights @ v

         # reshape it back before final linear layer
         A = A.transpose(1, 2) # (N, T, h, d_k)
         A = A.contiguous().view(N, T, self.d_k * self.n_heads) # (N, T, h*d_k)

         # projection
         return self.fc(A)


In [3]:
class TransformerBlock(nn.Module):
     def __init__(self, d_k, d_model, n_heads, dropout_prob=0.1):
         super().__init__()
         self.layerNorm1 = nn.LayerNorm(d_model)
         self.layerNorm2 = nn.LayerNorm(d_model)
         self.multiHeadAttention = MultiheadAttention(d_k, d_model, n_heads)
         self.feedForward = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Linear(d_model * 4, d_model),
            nn.Dropout(p=dropout_prob)
         )
         self.dropout = nn.Dropout(p=dropout_prob)

     def forward(self, x, mask=None):
         #residual + multihead then LayerNormalization
         x = self.layerNorm1(x + self.multiHeadAttention(x, x, x, mask))
         #residual + ann then LayerNormalization
         x = self.layerNorm2(x + self.feedForward(x))
         x = self.dropout(x)
         return x

In [4]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout_prob)

    position = torch.arange(max_len).unsqueeze(1)
    exp_term = torch.arange(0, d_model, 2)
    #equivalent to 10000^(-2i/d_model) but numerically more stable
    div_term = torch.exp(exp_term * (-math.log(10000.0) / d_model))
    pe = torch.zeros(1, max_len, d_model)
    pe[0, :, 0::2] = torch.sin(position * div_term)
    pe[0, :, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, x):
    # x.shape: N x T x D
    x = x + self.pe[:, :x.size(1), :]
    return self.dropout(x)

In [5]:
class Encoder(nn.Module):
  def __init__(self,
               vocab_size,
               max_len,
               d_k,
               d_model,
               n_heads,
               n_layers,
               n_classes,
               dropout_prob):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding(d_model, max_len, dropout_prob)
    transformer_blocks = [
        TransformerBlock(
            d_k,
            d_model,
            n_heads,
            dropout_prob) for _ in range(n_layers)]
    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln = nn.LayerNorm(d_model)
    self.fc = nn.Linear(d_model, n_classes)

  def forward(self, x, mask=None):
    x = self.embedding(x)
    x = self.pos_encoding(x)
    for block in self.transformer_blocks:
      x = block(x, mask)

    # many-to-one (x has the shape N x T x D)
    # if classification we keep only first output
    # if many-to-many needs all outputs
    x = x[:, 0, :]

    x = self.ln(x)
    x = self.fc(x)
    return x

In [6]:
vocab_size = 20_000
max_len = 1024
d_k = 16
d_model = 64
n_heads = 4
n_layers = 2
n_classes = 5
dropout_prob = 0.1
model = Encoder(vocab_size, max_len, d_k, d_model, n_heads, n_layers, n_classes, dropout_prob)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


Encoder(
  (embedding): Embedding(20000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (layerNorm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layerNorm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (multiHeadAttention): MultiheadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (feedForward): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
    

In [8]:
x = np.random.randint(0, 20_000, size=(8, 512))
x_t = torch.tensor(x).to(device)

In [9]:
mask = np.ones((8, 512))
mask[:, 256:] = 0
mask_t = torch.tensor(mask).to(device)

In [10]:
y = model(x_t, mask_t)

In [11]:
y.shape

torch.Size([8, 5])

In [12]:
!pip install datasets

In [13]:
!pip install transformers[torch]

In [14]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [15]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [16]:
from datasets import load_dataset

In [17]:
raw_datasets = load_dataset("glue", "sst2")

In [18]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [19]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

In [20]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [21]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [23]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [25]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
valid_loader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=32,
    collate_fn=data_collator
)

In [26]:
# check how it works
for batch in train_loader:
  for k, v in batch.items():
    print("k:", k, "v.shape:", v.shape)
  break

k: labels v.shape: torch.Size([32])
k: input_ids v.shape: torch.Size([32, 31])
k: attention_mask v.shape: torch.Size([32, 31])


In [27]:
set(tokenized_datasets['train']['labels'])

{0, 1}

In [28]:
tokenizer.vocab_size

28996

In [29]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [30]:
model = Encoder(
    vocab_size=tokenizer.vocab_size,
    max_len=tokenizer.max_model_input_sizes[checkpoint],
    d_k=16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    n_classes=2,
    dropout_prob=0.1,
)
model.to(device)

Encoder(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (layerNorm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layerNorm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (multiHeadAttention): MultiheadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (feedForward): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
    

In [31]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [32]:
from datetime import datetime

In [33]:
# A function to encapsulate the training loop
def train(model, criterion, optimizer, train_loader, valid_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = 0
    n_train = 0
    for batch in train_loader:
      # move data to GPU
      batch = {k: v.to(device) for k, v in batch.items()}

      # zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs, batch['labels'])

      # Backward and optimize
      loss.backward()
      optimizer.step()

      train_loss += loss.item()*batch['input_ids'].size(0)
      n_train += batch['input_ids'].size(0)

    # Get average train loss
    train_loss = train_loss / n_train

    model.eval()
    test_loss = 0
    n_test = 0
    for batch in valid_loader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs, batch['labels'])
      test_loss += loss.item()*batch['input_ids'].size(0)
      n_test += batch['input_ids'].size(0)
    test_loss = test_loss / n_test

    # Save losses
    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
      Test Loss: {test_loss:.4f}, Duration: {dt}')

  return train_losses, test_losses

In [36]:
train_losses, test_losses = train(
    model, criterion, optimizer, train_loader, valid_loader, epochs=7)

Epoch 1/7, Train Loss: 0.1452,       Test Loss: 0.5891, Duration: 0:00:19.662511
Epoch 2/7, Train Loss: 0.1362,       Test Loss: 0.6532, Duration: 0:00:17.317688
Epoch 3/7, Train Loss: 0.1285,       Test Loss: 0.6140, Duration: 0:00:19.160003
Epoch 4/7, Train Loss: 0.1229,       Test Loss: 0.6329, Duration: 0:00:17.517559
Epoch 5/7, Train Loss: 0.1165,       Test Loss: 0.7093, Duration: 0:00:17.473000
Epoch 6/7, Train Loss: 0.1112,       Test Loss: 0.7055, Duration: 0:00:17.283235
Epoch 7/7, Train Loss: 0.1076,       Test Loss: 0.6283, Duration: 0:00:17.587388


In [37]:
# Accuracy

model.eval()
n_correct = 0.
n_total = 0.
for batch in train_loader:
  # Move to GPU
  batch = {k: v.to(device) for k, v in batch.items()}

  # Forward pass
  outputs = model(batch['input_ids'], batch['attention_mask'])

  # Get prediction
  # torch.max returns both max and argmax
  _, predictions = torch.max(outputs, 1)

  # update counts
  n_correct += (predictions == batch['labels']).sum().item()
  n_total += batch['labels'].shape[0]

train_acc = n_correct / n_total


n_correct = 0.
n_total = 0.
for batch in valid_loader:
  # Move to GPU
  batch = {k: v.to(device) for k, v in batch.items()}

  # Forward pass
  outputs = model(batch['input_ids'], batch['attention_mask'])

  # Get prediction
  # torch.max returns both max and argmax
  _, predictions = torch.max(outputs, 1)

  # update counts
  n_correct += (predictions == batch['labels']).sum().item()
  n_total += batch['labels'].shape[0]

test_acc = n_correct / n_total
print(f"Train acc: {train_acc:.4f}, Test acc: {test_acc:.4f}")


Train acc: 0.9807, Test acc: 0.8050
